# Telco Churn Prediction (Logistic Regression)

**Goal:** Predict customer churn (`churn` = 1) using a leakage-resistant feature set and evaluate the model with **ROC-AUC** and **PR-AUC**.

**Inputs (provided):**
- `telcodata_extract1.csv` (customer profile + usage/behavior)
- `telcodata_extract2.csv` (label `churn`, `upsell_xsell`)
- `telcodata_extract3.csv` (plan / device / lifecycle + call-center attributes)
- `Data_dictionary_clean_combined.csv` (variable types + labels)

**Outputs (generated by this notebook):**
- `Telco_Churn_LogReg_Analysis.xlsx` (data audit + model results)

> Note: We intentionally exclude variables that can cause **label leakage** (e.g., current-month charges, call-center intent fields).


In [ ]:
# =========================
# 0) Setup
# =========================
import os
import re
from dataclasses import dataclass
from typing import Dict, List, Tuple

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from IPython.display import display

from sklearn.metrics import (
    roc_auc_score,
    average_precision_score,
    confusion_matrix,
    precision_score,
    recall_score,
)

# Reproducibility
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

DATA_DIR = "/mnt/data"  # <- change if needed


## 1) Helper functions (load/merge, cleaning, feature selection, modeling)

In [ ]:
def _clean_numeric_strings(s: pd.Series) -> pd.Series:
    """Convert currency/commas/%/mixed strings to numeric; invalid -> NaN."""
    return pd.to_numeric(
        s.astype(str).str.replace(r"[^0-9\.\-]", "", regex=True),
        errors="coerce",
    )



def make_onehot_encoder(min_frequency: int = 200):
    """Compatibility helper: scikit-learn changed OneHotEncoder 'sparse' -> 'sparse_output'."""
    try:
        return OneHotEncoder(handle_unknown="ignore", min_frequency=min_frequency, sparse_output=True)
    except TypeError:
        # older scikit-learn
        return OneHotEncoder(handle_unknown="ignore", min_frequency=min_frequency, sparse=True)

def load_and_merge(data_dir: str) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """Load 3 extracts and merge into one modeling table + load data dictionary."""
    p1 = os.path.join(data_dir, "telcodata_extract1.csv")
    p2 = os.path.join(data_dir, "telcodata_extract2.csv")
    p3 = os.path.join(data_dir, "telcodata_extract3.csv")
    ddp = os.path.join(data_dir, "Data_dictionary_clean_combined.csv")

    df1 = pd.read_csv(p1, low_memory=False)
    df2 = pd.read_csv(p2)
    df3 = pd.read_csv(p3)
    dd = pd.read_csv(ddp)

    # Align key name
    df3 = df3.rename(columns={"cust_id": "Customer_ID"})

    # Merge all
    df = df1.merge(df2, on="Customer_ID").merge(df3, on="Customer_ID")
    return df, dd


def prep_features(df: pd.DataFrame, dd: pd.DataFrame) -> Tuple[pd.DataFrame, Dict[str, str], List[str]]:
    """Clean numeric fields (based on data dictionary) and return type maps."""
    type_map = dd.set_index("variable")["type"].to_dict()
    label_map = dd.set_index("variable")["label"].to_dict()

    num_cols = [c for c in df.columns if type_map.get(c) == "Num"]

    out = df.copy()
    for c in num_cols:
        if c in {"Customer_ID", "churn", "upsell_xsell"}:
            continue
        if out[c].dtype == "object":
            out[c] = _clean_numeric_strings(out[c])

    return out, label_map, num_cols


def build_dataset(df: pd.DataFrame, dd: pd.DataFrame) -> Tuple[pd.DataFrame, List[str], List[str], List[str], pd.DataFrame]:
    """Select a leakage-resistant feature set for churn modeling."""
    dfc, label_map, _num_cols = prep_features(df, dd)

    target = "churn"
    exclude_always = {"Customer_ID", target, "upsell_xsell"}
    all_features = [c for c in dfc.columns if c not in exclude_always]

    # 1) Drop obvious "current month" fields (often leak churn status)
    leak_cols = [
        "voice_tot_bill_mou_curr",
        "tot_voice_chrgs_curr",
        "mb_data_usg_m01",
        "mb_data_usg_roamm01",
        "tot_mb_data_curr",
        "tot_mb_data_roam_curr",
        "data_prem_chrgs_curr",
    ]
    all_features = [c for c in all_features if c not in leak_cols]

    # 2) Remove call-center intent fields (usually occur near/after churn decision)
    call_center_like = {
        "nbr_contacts",
        "calls_TS_acct",
        "calls_care_acct",
        "calls_care_3mavg_acct",
        "calls_care_6mavg_acct",
        "res_calls_3mavg_acct",
        "res_calls_6mavg_acct",
        "last_rep_sat_score",
        "network_mention",
        "service_mention",
        "price_mention",
        "call_center",
        "issue_level1",
        "issue_level2",
        "call_category_1",
        "call_category_2",
        "resolution",
    }
    all_features = [c for c in all_features if c not in call_center_like]

    # 3) Remove suspension/status fields (often label-adjacent)
    suspension_like = {"count_of_suspensions_6m", "avg_days_susp"}
    all_features = [c for c in all_features if c not in suspension_like]

    # 4) Remove sensitive demographic proportions (fairness/compliance)
    sensitive_cs = {
        "cs_hispanic",
        "cs_caucasian",
        "cs_afr_amer",
        "cs_other",
        "cs_ttl_male",
        "cs_ttl_female",
    }
    all_features = [c for c in all_features if c not in sensitive_cs]

    # 5) Remove ID-like/high-cardinality geography represented as numbers
    all_features = [c for c in all_features if c not in {"zipcode_primary", "state"}]

    # Identify numeric vs categorical by dtype (after cleaning)
    num_features = [c for c in all_features if pd.api.types.is_numeric_dtype(dfc[c])]
    cat_features = [c for c in all_features if c not in num_features]

    # Optional: city is extremely high-cardinality; drop for interpretability
    cat_features = [c for c in cat_features if c != "city"]

    all_features = num_features + cat_features

    # Build exclusion report
    excluded = sorted(list(set([c for c in dfc.columns if c not in exclude_always]) - set(all_features) - set(leak_cols)))
    excluded_df = pd.DataFrame(
        {
            "variable": excluded,
            "label": [label_map.get(v, v) for v in excluded],
            "reason": [
                "Leakage / not available before churn"
                if v in (set(leak_cols) | call_center_like | suspension_like)
                else "Removed for fairness/compliance"
                if v in sensitive_cs
                else "ID-like/high-cardinality"
                if v in {"zipcode_primary", "state", "city"}
                else "Excluded"
                for v in excluded
            ],
        }
    )

    return dfc, all_features, num_features, cat_features, excluded_df


def fit_logistic_regression(
    dfc: pd.DataFrame,
    features: List[str],
    num_features: List[str],
    cat_features: List[str],
    random_state: int = 42,
) -> Tuple[Pipeline, pd.DataFrame, pd.Series, pd.Series]:
    """Fit a regularized logistic regression model with preprocessing."""
    X = dfc[features]
    y = dfc["churn"].astype(int)

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.30, random_state=random_state, stratify=y
    )

    numeric_transformer = Pipeline(
        steps=[
            ("imputer", SimpleImputer(strategy="median")),
            ("scaler", StandardScaler()),
        ]
    )

    categorical_transformer = Pipeline(
        steps=[
            ("imputer", SimpleImputer(strategy="most_frequent")),
            ("onehot", make_onehot_encoder(min_frequency=200)),
        ]
    )

    preprocess = ColumnTransformer(
        transformers=[
            ("num", numeric_transformer, num_features),
            ("cat", categorical_transformer, cat_features),
        ],
        remainder="drop",
    )

    model = LogisticRegression(
        solver="liblinear",
        penalty="l2",
        class_weight="balanced",
        max_iter=300,
    )

    clf = Pipeline(steps=[("preprocess", preprocess), ("model", model)])
    clf.fit(X_train, y_train)

    proba = pd.Series(clf.predict_proba(X_test)[:, 1], index=y_test.index)
    return clf, X_test, y_test, proba


@dataclass
class ModelOutputs:
    roc_auc: float
    pr_auc: float
    baseline_rate: float
    top10_threshold: float
    top10_precision: float
    top10_recall: float
    confusion: np.ndarray
    decile_table: pd.DataFrame
    coef_table: pd.DataFrame
    data_quality: pd.DataFrame
    excluded: pd.DataFrame


def summarize_model(
    clf: Pipeline,
    dfc: pd.DataFrame,
    features: List[str],
    excluded_df: pd.DataFrame,
    random_state: int = 42,
) -> ModelOutputs:
    """Compute AUCs, lift/deciles, coefficients, and data-quality tables."""
    X = dfc[features]
    y = dfc["churn"].astype(int)

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.30, random_state=random_state, stratify=y
    )
    clf.fit(X_train, y_train)
    score = clf.predict_proba(X_test)[:, 1]

    roc_auc = float(roc_auc_score(y_test, score))
    pr_auc = float(average_precision_score(y_test, score))
    baseline = float(y_test.mean())

    # Top 10% threshold
    top10 = float(np.quantile(score, 0.9))
    pred = (score >= top10).astype(int)
    cm = confusion_matrix(y_test, pred)
    prec = float(precision_score(y_test, pred))
    rec = float(recall_score(y_test, pred))

    # Decile lift table (D10 highest risk)
    t = pd.DataFrame({"y": y_test.values, "score": score})
    t["decile"] = pd.qcut(t["score"], 10, labels=False, duplicates="drop")
    dec = (
        t.groupby("decile")
        .agg(
            customers=("y", "size"),
            churners=("y", "sum"),
            churn_rate=("y", "mean"),
            avg_score=("score", "mean"),
        )
        .reset_index()
        .sort_values("decile", ascending=False)
        .reset_index(drop=True)
    )
    dec["label"] = [f"D{i}" for i in range(10, 0, -1)]
    dec["lift"] = dec["churn_rate"] / baseline
    dec["cum_churners"] = dec["churners"].cumsum()
    dec["capture_rate"] = dec["cum_churners"] / dec["churners"].sum()
    decile_table = dec[["label", "customers", "churners", "churn_rate", "lift", "capture_rate", "avg_score"]]

    # Coefficients (standardized / one-hot)
    feature_names = clf.named_steps["preprocess"].get_feature_names_out()
    coefs = clf.named_steps["model"].coef_.ravel()
    coef_df = pd.DataFrame({"feature": feature_names, "coef": coefs})
    coef_df["odds_ratio"] = np.exp(coef_df["coef"])
    coef_df["abs_coef"] = coef_df["coef"].abs()
    coef_df = coef_df.sort_values("abs_coef", ascending=False).reset_index(drop=True)

    # Data quality
    dq = pd.DataFrame(
        {
            "variable": features,
            "dtype": [str(dfc[c].dtype) for c in features],
            "missing_pct": [float(dfc[c].isna().mean()) for c in features],
            "n_unique": [int(dfc[c].nunique(dropna=True)) for c in features],
        }
    ).sort_values("missing_pct", ascending=False)

    return ModelOutputs(
        roc_auc=roc_auc,
        pr_auc=pr_auc,
        baseline_rate=baseline,
        top10_threshold=top10,
        top10_precision=prec,
        top10_recall=rec,
        confusion=cm,
        decile_table=decile_table,
        coef_table=coef_df,
        data_quality=dq,
        excluded=excluded_df,
    )


def build_excel_report(out_path: str, dd: pd.DataFrame, outputs: ModelOutputs, sample: pd.DataFrame) -> None:
    """Create a multi-sheet Excel report (no external dependencies beyond openpyxl)."""
    from openpyxl import Workbook
    from openpyxl.utils import get_column_letter
    from openpyxl.styles import Alignment, Font, PatternFill
    from openpyxl.worksheet.table import Table, TableStyleInfo

    header_fill = PatternFill("solid", fgColor="0B1F3A")
    header_font = Font(color="FFFFFF", bold=True)
    center = Alignment(horizontal="center", vertical="center", wrap_text=True)

    def style_header(ws, row: int) -> None:
        for cell in ws[row]:
            cell.fill = header_fill
            cell.font = header_font
            cell.alignment = center

    def add_table(ws, ref: str, name: str) -> None:
        tab = Table(displayName=name, ref=ref)
        tab.tableStyleInfo = TableStyleInfo(name="TableStyleMedium9", showRowStripes=True)
        ws.add_table(tab)

    wb = Workbook()
    wb.remove(wb.active)

    # README
    ws = wb.create_sheet("README")
    ws["A1"] = "Telco churn — Logistic Regression report"
    ws["A1"].font = Font(size=16, bold=True, color="0B1F3A")
    ws["A3"] = f"ROC-AUC: {outputs.roc_auc:.3f} | PR-AUC: {outputs.pr_auc:.3f} | Baseline churn: {outputs.baseline_rate:.2%}"
    ws["A4"] = (
        f"Top 10% risk threshold: {outputs.top10_threshold:.3f} | "
        f"Precision: {outputs.top10_precision:.1%} | Recall: {outputs.top10_recall:.1%}"
    )
    ws.column_dimensions["A"].width = 120

    # Data dictionary
    ws = wb.create_sheet("DATA_DICTIONARY")
    ws.append(list(dd.columns))
    style_header(ws, 1)
    for r in dd.itertuples(index=False):
        ws.append(list(r))
    add_table(ws, f"A1:G{ws.max_row}", "DataDict")

    # Data quality
    ws = wb.create_sheet("DATA_QUALITY")
    ws.append(list(outputs.data_quality.columns))
    style_header(ws, 1)
    for r in outputs.data_quality.itertuples(index=False):
        ws.append(list(r))
    add_table(ws, f"A1:D{ws.max_row}", "DataQuality")

    # Excluded
    ws = wb.create_sheet("EXCLUDED")
    ws.append(list(outputs.excluded.columns))
    style_header(ws, 1)
    for r in outputs.excluded.itertuples(index=False):
        ws.append(list(r))
    add_table(ws, f"A1:C{ws.max_row}", "Excluded")

    # Performance
    ws = wb.create_sheet("PERFORMANCE")
    ws.append(["metric", "value"])
    style_header(ws, 1)
    rows = [
        ("ROC_AUC", outputs.roc_auc),
        ("PR_AUC", outputs.pr_auc),
        ("Baseline churn", outputs.baseline_rate),
        ("Top10 threshold", outputs.top10_threshold),
        ("Top10 precision", outputs.top10_precision),
        ("Top10 recall", outputs.top10_recall),
    ]
    for k, v in rows:
        ws.append([k, float(v)])

    ws.append([])
    ws.append(["Confusion matrix (Top10)", "Pred 0", "Pred 1"])
    style_header(ws, ws.max_row)
    ws.append(["Actual 0", int(outputs.confusion[0, 0]), int(outputs.confusion[0, 1])])
    ws.append(["Actual 1", int(outputs.confusion[1, 0]), int(outputs.confusion[1, 1])])

    # Deciles
    ws = wb.create_sheet("DECILE_LIFT")
    ws.append(list(outputs.decile_table.columns))
    style_header(ws, 1)
    for r in outputs.decile_table.itertuples(index=False):
        ws.append(list(r))
    add_table(ws, f"A1:G{ws.max_row}", "DecileLift")

    # Coefs (top 50)
    ws = wb.create_sheet("COEFFICIENTS")
    top = outputs.coef_table.head(50).copy()
    ws.append(list(top.columns))
    style_header(ws, 1)
    for r in top.itertuples(index=False):
        ws.append(list(r))
    add_table(ws, f"A1:D{ws.max_row}", "Coefs")

    # Sample
    ws = wb.create_sheet("SAMPLE_CLEAN")
    ws.append(list(sample.columns))
    style_header(ws, 1)
    for r in sample.itertuples(index=False):
        ws.append(list(r))
    last_col = get_column_letter(sample.shape[1])
    add_table(ws, f"A1:{last_col}{ws.max_row}", "Sample")

    wb.save(out_path)


## 2) Load data, build features, train Logistic Regression, compute AUC

In [ ]:
# Load + merge
df_raw, dd = load_and_merge(DATA_DIR)

print("Merged dataset shape:", df_raw.shape)
print("Target distribution (churn):")
display(df_raw["churn"].value_counts(dropna=False).to_frame("count"))
print("Churn rate:", df_raw["churn"].mean())

# Build modeling dataset (feature selection + leakage control)
dfc, features, num_features, cat_features, excluded_df = build_dataset(df_raw, dd)

print("\nSelected features:", len(features))
print(" - numeric:", len(num_features))
print(" - categorical:", len(cat_features))
print("\nExcluded variables (count):", excluded_df.shape[0])
display(excluded_df.head(10))

# Fit model + evaluate
clf, X_test, y_test, proba_test = fit_logistic_regression(dfc, features, num_features, cat_features, random_state=RANDOM_STATE)

roc_auc = roc_auc_score(y_test, proba_test)
pr_auc  = average_precision_score(y_test, proba_test)

print(f"ROC-AUC (test): {roc_auc:.3f}")
print(f"PR-AUC  (test): {pr_auc:.3f}")


## 3) Business view: Top-decile targeting, lift & capture

In [ ]:
outputs = summarize_model(clf, dfc, features, excluded_df, random_state=RANDOM_STATE)

print("Baseline churn rate:", f"{outputs.baseline_rate:.2%}")
print("Top 10% threshold:", f"{outputs.top10_threshold:.3f}")
print("Top 10% precision:", f"{outputs.top10_precision:.1%}")
print("Top 10% recall:", f"{outputs.top10_recall:.1%}")
print("\nConfusion matrix (Top10 threshold):")
display(pd.DataFrame(outputs.confusion, index=["Actual 0","Actual 1"], columns=["Pred 0","Pred 1"]))

print("\nDecile lift table (D10 = highest risk):")
display(outputs.decile_table)


## 4) Feature importance (coefficients & odds ratios)

> Logistic Regression is interpretable: coefficients show how features move churn odds (after preprocessing: numeric standardized, categorical one-hot).


In [ ]:
# Top drivers by absolute coefficient
display(outputs.coef_table.head(20))

# (Optional) show the strongest positive drivers only
top_pos = outputs.coef_table.sort_values("coef", ascending=False).head(15)
top_neg = outputs.coef_table.sort_values("coef", ascending=True).head(15)

print("Strongest POSITIVE drivers (increase churn odds):")
display(top_pos)

print("Strongest NEGATIVE drivers (decrease churn odds):")
display(top_neg)


## 5) Export Excel report (deliverable)

This creates a multi-sheet Excel file you can attach to your submission.


In [ ]:
OUT_XLSX = os.path.join(DATA_DIR, "Telco_Churn_LogReg_Analysis.xlsx")

# Include a small clean sample for review (first 200 rows)
sample_cols = ["Customer_ID"] + features + ["churn"]
sample = dfc[sample_cols].head(200).copy()

build_excel_report(OUT_XLSX, dd, outputs, sample)

print("Saved:", OUT_XLSX)


## 6) (Optional) Save churn risk scores for all customers

In [ ]:
# Fit on full dataset to score everyone (for deployment / campaign list)
X_full = dfc[features]
y_full = dfc["churn"].astype(int)

clf.fit(X_full, y_full)
scores = clf.predict_proba(X_full)[:, 1]

scored = pd.DataFrame({
    "Customer_ID": dfc["Customer_ID"].values,
    "churn": y_full.values,
    "score": scores
}).sort_values("score", ascending=False)

OUT_SCORES = os.path.join(DATA_DIR, "Telco_Churn_Scored.csv")
scored.to_csv(OUT_SCORES, index=False)

print("Saved:", OUT_SCORES)
display(scored.head(10))
